In [28]:
import numpy as np


In [44]:
class QuantumNetworkTensor:
    def __init__(self, nodes, maxQubits, hardLinks, cutoffAge):
        self.nodes = nodes
        self.maxQubits = maxQubits
        self.hardLinks = hardLinks
        self.cutoffAge = cutoffAge
        self.currentLinks = None

    def createEmptyHardLinksTensor(self):
        tensor = np.full((self.nodes, self.nodes, self.maxQubits), 0)
        
        for source, targets in self.hardLinks.items():
            for target in targets:
                s, t = source, target
                idx = 0
                while idx < self.maxQubits and tensor[s, t, idx]:
                    idx += 1
                if idx < self.maxQubits:
                    tensor[s, t, idx] = 1
                    tensor[t, s, idx] = 1
        self.hardLinksTensor = tensor    
    
    def resetCurrentLinks(self):
        self.currentLinks = np.full((self.nodes, self.nodes, self.maxQubits), 0)
        
    def generateEntanglement(self, node1, node2):
        # Check if there's a hard link between the nodes
        if not self.hardLinksTensor[node1, node2, :].any():
            print("No hard link between the nodes")
            return False
        
        # Find first available qubit slot in current links
        for idx in range(self.maxQubits):
            if not self.currentLinks[node1, node2, idx]:
                self.currentLinks[node1, node2, idx] = 0
                self.currentLinks[node2, node1, idx] = 0
                return True
        print("No available qubit slots")
        return False  # No available qubit slots
    
    def ageAndDiscardEntanglements(self):
        # Age all entanglement slots by 1 and remove any that have reached cutoffAge
        for i in range(self.nodes):
            for j in range(i + 1, self.nodes):  # Only iterate through upper triangle
                for k in range(self.maxQubits):
                    if self.currentLinks[i, j, k] > 0:  # If there's an active entanglement
                        self.currentLinks[i, j, k] += 1  # Age it by 1
                        self.currentLinks[j, i, k] += 1  # Mirror the aging
                        # If it reached cutoff age, remove it
                        if self.currentLinks[i, j, k] >= self.cutoffAge:
                            self.currentLinks[i, j, k] = 0
                            self.currentLinks[j, i, k] = 0
    


In [43]:
# Example hard links:
hardLinks = {
    0: [2],
    1: [2,],
    2: [0, 1]
}

myNetwork = QuantumNetworkTensor(nodes=3, maxQubits=2, hardLinks=hardLinks)
myNetwork.createEmptyHardLinksTensor()
myNetwork.resetCurrentLinks()
myNetwork.generateEntanglement(0, 2)
myNetwork.generateEntanglement(0, 2)
myNetwork.generateEntanglement(0, 2)
print('done')

No available qubit slots
done
